In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("bin_futures_historical_pairs_with_spreads_normalized.csv", sep = ";")

In [3]:
df

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spreadNorm,TRX_XRP_alpha,TRX_XRP_beta,TRX_XRP_corr,TRX_XRP_pval,BTC_TRX_spreadNorm,BTC_TRX_alpha,BTC_TRX_beta,BTC_TRX_corr,BTC_TRX_pval
0,4.423169,-0.818257,2.166078,0.021370,2.136176,3.486794,6.072814,6.358864,11.013869,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.422809,-0.819164,2.164012,0.020195,2.138418,3.485937,6.071938,6.357825,11.014249,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.421368,-0.819845,2.163553,0.018527,2.136294,3.485630,6.070992,6.357634,11.012786,-2.016504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.422929,-0.818937,2.167223,0.020097,2.136176,3.486304,6.072422,6.357409,11.013483,-2.015528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.422569,-0.818937,2.166651,0.019508,2.135349,3.485937,6.071476,6.357201,11.012911,-2.015303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525595,5.100171,-0.383899,1.676592,-1.122700,1.456986,3.038313,5.901403,6.394894,11.452122,-1.759563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525596,5.100232,-0.383899,1.675488,-1.122393,1.457452,3.039271,5.902415,6.394978,11.452460,-1.759157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525597,5.101085,-0.383753,1.676143,-1.121779,1.458150,3.039845,5.902469,6.395746,11.452573,-1.758750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525598,5.101329,-0.383753,1.676405,-1.121165,1.457917,3.039893,5.902415,6.395762,11.452698,-1.758808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(525600, 392)

In [ ]:
import pandas as pd
import numpy as np
import ta
import os

# Feature functions
def ewma_volatility(series, lambda_=0.94):
    """Exponentially weighted volatility"""
    returns = series.pct_change().dropna()
    return returns.ewm(alpha=1-lambda_).std()

def kalman_ma(series, window=20):
    """Simple rolling mean as placeholder for Kalman filter"""
    return series.rolling(window=window, min_periods=1).mean()

def compute_features_day(df_day):
    """
    Compute all features for a single day dataframe.
    Only for relevant assets and spreads (non-NA normalized spreads)
    """
    out = df_day.copy()
    
    # Identify spread columns for today that are not NA
    spread_cols = [col for col in df_day.columns if col.endswith("_spreadNorm") and df_day[col].notna().any()]
    
    # Identify all assets involved today
    assets = set()
    for col in spread_cols:
        parts = col.split("_")
        sym1, sym2 = parts[0], parts[1]
        assets.add(sym1)
        assets.add(sym2)
    
    price_cols = [f"{asset}_close" for asset in assets if f"{asset}_close" in df_day.columns]


    # Price-based features
    for col in price_cols:
        prices = df_day[col].astype(float)
        # MACD
        out[f"{col}Macd"] = ta.trend.MACD(prices, window_slow=14, window_fast=5).macd()
        # RSI
        out[f"{col}Rsi"] = ta.momentum.RSIIndicator(prices, window=20).rsi()
        # Kalman MA
        out[f"{col}Kalman"] = kalman_ma(prices, window=20)
        # EWMA volatility
        out[f"{col}EwmaVol"] = ewma_volatility(prices)
        # Bias (price - dynamic MA)
        out[f"{col}Bias"] = prices - out[f"{col}Kalman"]
        # Signs: ratio of positive vs negative returns over 10-min rolling window
        returns = prices.pct_change()
        out[f"{col}Signs"] = returns.rolling(window=10, min_periods=1).apply(lambda x: (x>0).sum()/max(1,len(x)), raw=False)
        # Stochastic RSI (simplified)
        rsi = out[f"{col}Rsi"]
        out[f"{col}StochRsi"] = (rsi - rsi.rolling(20,min_periods=1).min()) / (rsi.rolling(20,min_periods=1).max() - rsi.rolling(20,min_periods=1).min() + 1e-12)
        # Upper/Lower shadow proxies
        out[f"{col}UpperShadow"] = prices - prices.rolling(2).min()
        out[f"{col}LowerShadow"] = prices.rolling(2).max() - prices
    
    # Spread-based features
    for col in spread_cols:
        s = df_day[col].astype(float)
        out[col] = s
        out[f"{col}Vol"] = ewma_volatility(s)
        out[f"{col}Kalman"] = kalman_ma(s, window=20)
        # Moving average of spread
        out[f"{col}Ma"] = s.rolling(window=20, min_periods=1).mean()

    return out

# Main loop for all days
def compute_all_features(df):
    """
    Compute features day by day and save each day's df separately.
    df: full dataframe with timestamp, log prices, normalized spreads
    """
    df["date"] = df["timestamp"].dt.date
    feature_dfs = []
    
    for day, df_day in df.groupby("date"):
        print(f"Processing {day} ...")
        day_feats = compute_features_day(df_day)
        feature_dfs.append(day_feats)
    
    # Combine all days
    full_df = pd.concat(feature_dfs).sort_index()
    return full_df



In [7]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [8]:
features_df = compute_all_features(df)

Processing 2024-05-01 ...
Processing 2024-05-02 ...
Processing 2024-05-03 ...
Processing 2024-05-04 ...
Processing 2024-05-05 ...
Processing 2024-05-06 ...
Processing 2024-05-07 ...
Processing 2024-05-08 ...
Processing 2024-05-09 ...
Processing 2024-05-10 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Vol"] = ewma_volatility(s)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Kalman"] = kalman_ma(s, window=20)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

Processing 2024-05-11 ...
Processing 2024-05-12 ...
Processing 2024-05-13 ...
Processing 2024-05-14 ...
Processing 2024-05-15 ...
Processing 2024-05-16 ...
Processing 2024-05-17 ...
Processing 2024-05-18 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Vol"] = ewma_volatility(s)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Kalman"] = kalman_ma(s, window=20)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

Processing 2024-05-19 ...
Processing 2024-05-20 ...


...

Processing 2025-04-26 ...
Processing 2025-04-27 ...
Processing 2025-04-28 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Vol"] = ewma_volatility(s)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Kalman"] = kalman_ma(s, window=20)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

Processing 2025-04-29 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Vol"] = ewma_volatility(s)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Kalman"] = kalman_ma(s, window=20)
C:\Users\isaen\AppData\Local\Temp\ipykernel_17240\2183714022.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

Processing 2025-04-30 ...


In [9]:
features_df["BTC_TRX_spreadNormKalman"].isna().sum()

514080

In [10]:
features_df["BTC_close"]

0         11.013869
1         11.014249
2         11.012786
3         11.013483
4         11.012911
            ...    
525595    11.452122
525596    11.452460
525597    11.452573
525598    11.452698
525599    11.452380
Name: BTC_close, Length: 525600, dtype: float64

In [11]:
len(set(features_df["date"])) == 365

True

In [16]:
features_df.shape

(525600, 740)

In [13]:
cols_to_drop = [col for col in features_df.columns if col.endswith("_spread")]
len(cols_to_drop)

61

In [14]:
features_df = features_df.drop(columns = cols_to_drop, axis = 1)

In [ ]:
# features_df.to_csv("bin_futures_full_features.csv", index = False)

In [15]:
# Check that no column is NA everywhere
for col in features_df.columns:
    if features_df[col].isna().all():
        print(f"Column {col} is NA everywhere!")
# at least no NA everywhere


# Funding rate merging

In [17]:
features_df = features_df.rename(columns = {"timestamp": "datetime"})

In [18]:
import pandas as pd
from pathlib import Path

data_dir = Path(".")

symbols = [
    "AAVEUSDT", "ADAUSDT", "APTUSDT", "ARBUSDT", "ATOMUSDT",
    "AVAXUSDT", "BCHUSDT", "BNBUSDT", "BTCUSDT", "DOGEUSDT",
    "DOTUSDT", "ENAUSDT", "ETCUSDT", "ETHUSDT", "HBARUSDT",
    "LINKUSDT", "LTCUSDT", "NEARUSDT", "SUIUSDT",
    "TONUSDT", "TRXUSDT", "UNIUSDT", "WLDUSDT", "XLMUSDT", "XRPUSDT"
]

# --- Prepare base dataframe ---
features_df["datetime"] = pd.to_datetime(features_df["datetime"])
features_df = features_df.sort_values("datetime").reset_index(drop=True)

# Helper: funding interval per symbol
def funding_interval(symbol):
    if symbol in ["TONUSDT", "ENAUSDT"]:
        return pd.Timedelta(hours=4)
    else:
        return pd.Timedelta(hours=8)

for sym in symbols:
    try:
        path = data_dir / f"{sym}_funding_bin_futures.parquet"
        funding_df = pd.read_parquet(path)
        funding_df["datetime"] = pd.to_datetime(funding_df["datetime"])
        funding_df = funding_df.sort_values("datetime").reset_index(drop=True)

        # Compute next funding timestamp for each record
        interval = funding_interval(sym)
        funding_df["next_funding_time"] = funding_df["datetime"].shift(-1)
        funding_df.loc[funding_df["next_funding_time"].isna(), "next_funding_time"] = (
            funding_df["datetime"] + interval
        )
        sym = sym.replace("USDT","")
        # Initialize columns
        rate_col = f"{sym}_funding"
        time_col = f"{sym}_fundingMinutesLeft"
        features_df[rate_col] = None
        features_df[time_col] = None

        for _, row in funding_df.iterrows():
            mask = (features_df["datetime"] >= row["datetime"]) & (features_df["datetime"] < row["next_funding_time"])
            minutes_left = ((row["next_funding_time"] - features_df.loc[mask, "datetime"]).dt.total_seconds() / 60).astype(int)
            features_df.loc[mask, rate_col] = row["funding_rate"]
            features_df.loc[mask, time_col] = minutes_left

        print(f"✅ Added {sym}: funding rate + minutes left (interval {interval.components.hours}h)")

    except Exception as e:
        print(f"⚠️ Error adding {sym}: {e}")

print("\n🎯 Done — funding rates and countdown columns added for all symbols!")


✅ Added AAVE: funding rate + minutes left (interval 8h)
✅ Added ADA: funding rate + minutes left (interval 8h)
✅ Added APT: funding rate + minutes left (interval 8h)
✅ Added ARB: funding rate + minutes left (interval 8h)
✅ Added ATOM: funding rate + minutes left (interval 8h)
✅ Added AVAX: funding rate + minutes left (interval 8h)
✅ Added BCH: funding rate + minutes left (interval 8h)
✅ Added BNB: funding rate + minutes left (interval 8h)
✅ Added BTC: funding rate + minutes left (interval 8h)
✅ Added DOGE: funding rate + minutes left (interval 8h)
✅ Added DOT: funding rate + minutes left (interval 8h)
✅ Added ENA: funding rate + minutes left (interval 4h)
✅ Added ETC: funding rate + minutes left (interval 8h)
✅ Added ETH: funding rate + minutes left (interval 8h)
✅ Added HBAR: funding rate + minutes left (interval 8h)
✅ Added LINK: funding rate + minutes left (interval 8h)
✅ Added LTC: funding rate + minutes left (interval 8h)
✅ Added NEAR: funding rate + minutes left (interval 8h)
✅ A

In [20]:
features_df.shape
# 25*10 (log prices) +61*8 (spread features)+ 25*2 (funding) + 2 (time)

(525600, 790)

In [22]:
features_df.to_csv("bin_futures_full_features.csv", index = False)